In [1]:
import pandas as pd
import numpy as np
# from numba import njit

import matplotlib.pyplot as plt
from matplotlib import cm
from numpy import linalg as LA
from tqdm import tqdm
# from numba import njit

from kernel import *

# from sklearn import svm

from kernel_functions import * # gram_phi, count_kuplet_k, count_kuplet_3
from preprocessing import preprocessing

In [2]:
# load all data as the numpy array type
#X = pd.read_csv('data/Xtr1_mat50.csv', sep=' ', header=None).values
X_raw0 = pd.read_csv('data/Xtr0.csv', sep= ' ', header = None).values.reshape((-1))
X_raw1 = pd.read_csv('data/Xtr1.csv', sep=' ', header=None).values.reshape((-1))
X_raw2 = pd.read_csv('data/Xtr2.csv', sep=' ', header=None).values.reshape((-1))

# transform to an array of string
X_valid0 = pd.read_csv('data/Xte0.csv', sep=' ', header=None).values.reshape((-1))
X_valid1 = pd.read_csv('data/Xte1.csv', sep=' ', header=None).values.reshape((-1))
X_valid2 = pd.read_csv('data/Xte1.csv', sep=' ', header=None).values.reshape((-1))


Y0 = pd.read_csv('data/Ytr0.csv', sep=',', header=0)['Bound'].values
Y1 = pd.read_csv('data/Ytr1.csv', sep=',', header=0)['Bound'].values
Y2 = pd.read_csv('data/Ytr2.csv', sep=',', header=0)['Bound'].values

#print('numerical features shape', X.shape)
#print('numerical features first row', X[0])
print('sequences shape: ', X_raw0.shape)
print('sequence first row: ', X_raw0[0])
print('labels shape', Y0.shape)

('sequences shape: ', (2000,))
('sequence first row: ', 'TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGGCGCCCTCTAGAGTTGTGTAAAGAAGTGGCGTCACCTCATTATAAATAAAAGGTTG')
('labels shape', (2000,))


In [21]:
from kernel import *
import multiprocessing # import Pool


def solve_svm_kernel(X_train, X_test, Y_train, Y_test, kernel='k_gram_gaussian', k=3, k2=6, lamb=0.1, gamma=0.1, kktreg=1e-9):
    """
    kernel in ['k_gram', 'k_gram_gaussian', 'k_substring', 'local_alignement', 'k_gram_concat']
    """
    assert kernel in ['k_gram', 'k_gram_gaussian', 'k_substring', 'local_alignement', 'k_gram_concat']
    
    if kernel in ['k_gram', 'k_gram_gaussian']:
        if k == 3:
            X_train_process = np.array([count_kuplet_3(x) for x in X_train])
            X_test_process = np.array([count_kuplet_3(x) for x in X_test])
        else:
            X_train_process = np.array([count_kuplet_k(x,k=k) for x in X_train])
            X_test_process = np.array([count_kuplet_k(x,k=k) for x in X_test])
        
        # Adding 1 for the sake of the bias
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
    
    if kernel=='k_gram_concat':
        print("building kernel...")
        X_train_process1 = np.array([count_kuplet_3(x) for x in X_train])
        X_train_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_train])
        X_train_process = np.concatenate((X_train_process1, X_train_process2), axis=1)
        X_test_process1 = np.array([count_kuplet_3(x) for x in X_test])
        X_test_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_test])
        X_test_process = np.concatenate((X_test_process1, X_test_process2), axis=1)
        
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
        
    if kernel in ["k_gram", 'k_gram_concat']:
        # Computing the Gram-Matrix
        K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)
        K_test = np.dot(X_test_process, np.transpose(X_train_process))
        
    if kernel=="k_gram_gaussian":
        # Computing the Gram-Matrix
        K = np.array([LA.norm(X_train_process - y, axis=1) for y in X_train_process])
        K = np.exp(-K/gamma)
        # K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train, lamb=lamb)
        K_test = np.array([LA.norm(X_train_process - y, axis=1) for y in X_test_process])
        print(K_test.shape)
        K_test = np.exp(-K_test/gamma)
    
    if kernel=='k_substring':
        # Computing the Gram-Matrix
        # X_train_process = X_train
        # X_test_process = X_test
        N_train = len(X_train)
        N_test = len(X_test)
        K = np.zeros((N_train, N_train))
        for i in tqdm(range(len(X_train))):# ,desc="Computing Traning Kernel":
            # print("building pool")
            # pool = multiprocessing.Pool(4)
            # print("build param")
            # param = [(X_train[i], X_train[j], k) for j in range(len(X_train))]
            # print("using pool")
            # K[i][:] = pool.map(compute_gap_kernel_param, [[X_train[i], X_train[j], k] for j in range(len(X_train))])
            # print(K_test[i][:])
            for j in range(i, len(X_train)):
                K[i][j] = K[j][i] = compute_gap_kernel(X_train[i], X_train[j], k)
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)
        K_test = np.zeros((N_test, N_train))
        for i in tqdm(range(len(X_test)), desc="Compution testing Kernel"):
            for j in range(len(X_train)):
                K_test[i][j] = compute_gap_kernel(X_test[i], X_train[j], k)
    
    if kernel=='local_alignement':
        # Computing the Gram-Matrix
        # X_train_process = X_train
        # X_test_process = X_test
        N_train = len(X_train)
        N_test = len(X_test)
        K = np.zeros((N_train, N_train))
        for i in tqdm(range(len(X_train))):# ,desc="Computing Traning Kernel":
            # print("building pool")
            # pool = multiprocessing.Pool(4)
            # print("build param")
            # param = [(X_train[i], X_train[j], k) for j in range(len(X_train))]
            # print("using pool")
            # K[i][:] = pool.map(compute_gap_kernel_param, [[X_train[i], X_train[j], k] for j in range(len(X_train))])
            # print(K_test[i][:])
            for j in range(i, len(X_train)):
                K[i][j] = K[j][i] = LA_kernel(X_train[i], X_train[j])
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)
        K_test = np.zeros((N_test, N_train))
        for i in tqdm(range(len(X_test)), desc="Compution testing Kernel"):
            for j in range(len(X_train)):
                K_test[i][j] = LA_kernel(X_test[i], X_train[j])
                
    # if kernel in ['k_gram', 'k_gram_gaussian']:
    n = K.shape[0]
    Y_predicted = np.dot(K_test, w[:n]) > 0.
    result = ((Y_test+1.)/ 2. == np.transpose(Y_predicted))
    Y_predicted_train = np.dot(K, w[:n]) > 0.
    result_train = ((Y_train+1)/ 2 == np.transpose(Y_predicted_train))
    # print(Y_predicted)
    if np.alltrue(Y_predicted):
        print("Toute les valeurs sont TRUE")
    if np.alltrue(Y_predicted==False):
        print("Toute les valeurs sont FALSE")
    return np.mean(result), np.mean(result_train)
             


In [ ]:
def edit_distance(str1, str2):
    """
    Compute the Edit Distance between str1 and str2.
    Return the number of insertion/deletion/substitution
    """
    n = len(str1)
    m = len(str2)
    table_str = np.zeros((n+1,m+1))

    table_str[0, :]=np.arange(m+1)
    table_str[:, 0] = np.arange(n+1)

    for i in range(1, n+1):
        for j in range(1, m+1):
            if(str1[i-1] == str2[j-1]):
                table_str[i, j] = min(table_str[i-1, j] + 1, table_str[i, j-1] + 1, table_str[i-1, j-1]) 
            else:
                table_str[i, j] = min(table_str[i-1, j] + 1, table_str[i, j-1] + 1, table_str[i-1, j-1]+2) 
                
    return table_str

X1 = 'ACCCTG'
X2 = 'AGGTC'
print(edit_distance(X1, X2))

In [12]:
# With Cross validation

X_train0, Y_train0, X_test0, Y_test0 = preprocessing(X_raw0, Y0)
X_train1, Y_train1, X_test1, Y_test1 = preprocessing(X_raw1, Y1)
X_train2, Y_train2, X_test2, Y_test2 = preprocessing(X_raw2, Y2)
print('train shape', X_train0.shape)
print('test shape', X_test0.shape)

print('train shape', X_train1.shape)
print('test shape', X_test1.shape)

print('train shape', X_train2.shape)
print('test shape', X_test2.shape)


('train shape', (1600,))
('test shape', (400,))
('train shape', (1600,))
('test shape', (400,))
('train shape', (1600,))
('test shape', (400,))


In [35]:
acc_test0, acc_train0 = solve_svm_kernel(X_train0, 
                                       X_test0, 
                                       Y_train0, 
                                       Y_test0, 
                                       kernel='k_gram_concat', 
                                       k=3, lamb=0.03, gamma=0.05)

print('accuracy for train : {}'.format(acc_train0))
print('accuracy for test : {}'.format(acc_test0))

building kernel...
     pcost       dcost       gap    pres   dres
 0:  7.9160e-01  1.8083e+00  3e+03  1e+00  2e+05
 1:  1.7616e+00 -4.8617e+01  5e+01  2e-02  3e+03
 2:  1.4040e+00 -6.4072e+00  8e+00  3e-03  4e+02
 3:  1.1340e+00 -3.6882e-01  2e+00  4e-04  5e+01
 4:  5.5260e-01  3.4627e-01  2e-01  8e-10  2e-08
 5:  4.1338e-01  3.7025e-01  4e-02  6e-10  1e-08
 6:  3.8571e-01  3.7907e-01  7e-03  5e-10  2e-09
 7:  3.8138e-01  3.8080e-01  6e-04  5e-10  7e-10
 8:  3.8101e-01  3.8097e-01  4e-05  5e-10  2e-10
 9:  3.8098e-01  3.8098e-01  1e-06  5e-10  5e-11
10:  3.8098e-01  3.8098e-01  4e-08  5e-10  1e-11
Optimal solution found.
accuracy for train : 0.951875
accuracy for test : 0.7275


In [38]:
acc_test1, acc_train1 = solve_svm_kernel(X_train1, 
                                       X_test1, 
                                       Y_train1, 
                                       Y_test1, 
                                       kernel='k_gram_concat', 
                                       k=3, lamb=0.03, gamma=1.)

print('accuracy for train : {}'.format(acc_train1))
print('accuracy for test : {}'.format(acc_test1))

building kernel...
     pcost       dcost       gap    pres   dres
 0:  4.7902e-01  1.4885e+00  3e+03  1e+00  7e+05
 1:  1.4595e+00 -4.4029e+01  5e+01  2e-02  1e+04
 2:  1.2391e+00 -5.2471e+00  6e+00  3e-03  1e+03
 3:  1.0478e+00 -1.0089e+00  2e+00  6e-04  3e+02
 4:  6.8265e-01  1.4102e-02  7e-01  1e-04  6e+01
 5:  2.8707e-01  2.0986e-01  8e-02  1e-05  5e+00
 6:  2.4925e-01  2.2941e-01  2e-02  2e-06  1e+00
 7:  2.3849e-01  2.3504e-01  3e-03  2e-07  1e-01
 8:  2.3655e-01  2.3606e-01  5e-04  3e-08  1e-02
 9:  2.3626e-01  2.3622e-01  3e-05  1e-09  7e-04
10:  2.3623e-01  2.3623e-01  1e-06  6e-10  2e-05
11:  2.3623e-01  2.3623e-01  4e-08  6e-10  3e-07
12:  2.3623e-01  2.3623e-01  1e-09  6e-10  3e-09
Optimal solution found.
accuracy for train : 0.98375
accuracy for test : 0.8875


In [37]:
acc_test2, acc_train2 = solve_svm_kernel(X_train2, 
                                       X_test2, 
                                       Y_train2, 
                                       Y_test2, 
                                       kernel='k_gram_concat', 
                                       k=3, lamb=0.01, gamma=1.)

print('accuracy for train : {}'.format(acc_train2))
print('accuracy for test : {}'.format(acc_test2))

building kernel...
     pcost       dcost       gap    pres   dres
 0:  1.0265e+00  2.0459e+00  3e+03  1e+00  1e+05
 1:  1.9919e+00 -4.9741e+01  5e+01  2e-02  2e+03
 2:  1.5420e+00 -8.4051e+00  1e+01  4e-03  4e+02
 3:  1.2886e+00 -5.6416e-01  2e+00  5e-04  4e+01
 4:  6.9607e-01  4.2928e-01  3e-01  9e-10  2e-08
 5:  5.0317e-01  4.6156e-01  4e-02  7e-10  1e-08
 6:  4.7737e-01  4.7185e-01  6e-03  5e-10  3e-09
 7:  4.7401e-01  4.7345e-01  6e-04  5e-10  9e-10
 8:  4.7367e-01  4.7362e-01  5e-05  5e-10  3e-10
 9:  4.7364e-01  4.7364e-01  2e-06  5e-10  7e-11
10:  4.7364e-01  4.7364e-01  9e-08  5e-10  2e-11
Optimal solution found.
accuracy for train : 0.921875
accuracy for test : 0.6025


## Best parameters found for k_gram kernel
#### k = 3
* Set0: lambda=1e-3, gamma = 5 (gaussian), score = 0.68
* Set1: lambda=0.01, score = 
* Set2: lambda= 1e-5, score = 

#### k = 6

* Set0: lamda=0.1 , score = 0.73 (on test)
* Set1: lambda=0.01, score= 0.85 (on test)
* Set2: lambda= 1e-1, score = 0.6875 (on test)

### k=3 et k=6 (concatenate)

* Set0: lamda=0.04, score = 0.7425 (on test)
* Set1: lambda=0.05, score = 0.89 (on test)


In [6]:
def solve_svm_test(X_train, X_test, Y_train, kernel='k_gram_gaussian', k=3, k2=6, lamb=0.1, gamma=0.1, kktreg=1e-9):
    """
    kernel in [k_gram, k_gram_gaussian, k_gram_concat]
    """
    Y_train_process = (Y_train-0.5) * 2
    if kernel in ['k_gram', 'k_gram_gaussian']:
        if k == 3:
            X_train_process = np.array([count_kuplet_3(x) for x in X_train])
            X_test_process = np.array([count_kuplet_3(x) for x in X_test])
        else:
            X_train_process = np.array([count_kuplet_k(x,k=k) for x in X_train])
            X_test_process = np.array([count_kuplet_k(x,k=k) for x in X_test])
        
        # Adding 1 for the sake of the bias
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
    
    if kernel=='k_gram_concat':
        print("building kernel...")
        X_train_process1 = np.array([count_kuplet_3(x) for x in X_train])
        X_train_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_train])
        X_train_process = np.concatenate((X_train_process1, X_train_process2), axis=1)
        X_test_process1 = np.array([count_kuplet_3(x) for x in X_test])
        X_test_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_test])
        X_test_process = np.concatenate((X_test_process1, X_test_process2), axis=1)
        
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
        # X_train_process = 
        
        
    if kernel in ["k_gram", 'k_gram_concat']:
        # Computing the Gram-Matrix
        K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train_process, lamb=lamb, kktreg=kktreg)
        K_test = np.dot(X_test_process, np.transpose(X_train_process))
    
    if kernel=="k_gram_gaussian":
        # Computing the Gram-Matrix
        K = np.array([LA.norm(X_train_process - y, axis=1) for y in X_train_process])
        K = np.exp(-K/gamma)
        # K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train_process, lamb=lamb)
        K_test = np.array([LA.norm(X_train_process - y, axis=1) for y in X_test_process])
        K_test = np.exp(-K_test/gamma)
    
    n = K.shape[0]
    Y_predicted = np.dot(K_test, w[:n]) > 0.
    Y_predicted = (Y_predicted + 0.)
    Y_predicted_train = np.dot(K, w[:n]) > 0.
    result_train = (Y_train == np.transpose(Y_predicted_train))
    print('accuracy on train : {}'.format(np.mean(result_train)))
    # result = ((Y_test+1)/ 2 == np.transpose(Y_predicted))
    
    return np.transpose(Y_predicted) # , np.transpose(Y_predicted_train)


In [5]:
print('train shape : {}'.format(X_raw0.shape))
print('train shape 2 : {}'.format(Y0.shape))
print('test shape : {}'.format(X_valid0.shape))

Y_predict0 = solve_svm_test(X_raw0, 
                           X_valid0, 
                           Y0, 
                           kernel='k_gram_concat', 
                           k=6, lamb=.1, gamma=5.)

train shape : (2000,)
train shape 2 : (2000,)
test shape : (1000,)
building kernel...
     pcost       dcost       gap    pres   dres
 0:  4.4991e+00  5.9937e+00  4e+03  1e+00  1e+05
 1:  4.7728e+00 -1.4998e+02  2e+02  6e-02  4e+03
 2:  2.1697e+00 -2.4667e+01  3e+01  9e-03  6e+02
 3:  1.6102e+00 -2.7395e+00  4e+00  1e-03  8e+01
 4:  1.1708e+00  3.6932e-01  8e-01  7e-05  6e+00
 5:  6.5294e-01  5.6559e-01  9e-02  8e-06  6e-01
 6:  6.1627e-01  5.9761e-01  2e-02  1e-06  1e-01
 7:  6.0795e-01  6.0526e-01  3e-03  2e-07  1e-02
 8:  6.0670e-01  6.0643e-01  3e-04  1e-08  9e-04
 9:  6.0656e-01  6.0655e-01  9e-06  6e-10  2e-05
10:  6.0656e-01  6.0656e-01  3e-07  7e-10  5e-07
11:  6.0656e-01  6.0656e-01  1e-08  7e-10  6e-09
Optimal solution found.
accuracy on train : 0.8255


In [9]:
X_train = X_raw0
X_test = X_valid0
k2= 6
X_train_process1 = np.array([count_kuplet_3(x) for x in X_train])
X_train_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_train])
X_train_process = np.concatenate((X_train_process1, X_train_process2), axis=1)


X_test_process1 = np.array([count_kuplet_3(x) for x in X_test])
X_test_process2 = np.array([count_kuplet_k(x, k=k2) for x in X_test])
X_test_process = np.concatenate((X_test_process1, X_test_process2), axis=1)
# print(X_train_process1.shape)

(2000, 64)


In [10]:
print(X_train_process2.shape)

(2000, 4096)


In [12]:
X_train_process = np.concatenate((X_train_process1, X_train_process2), axis=1)
print(X_train_process.shape)

(2000, 4160)


In [26]:
print('train shape : {}'.format(X_raw1.shape))
print('train shape 2 : {}'.format(Y1.shape))
print('test shape : {}'.format(X_valid1.shape))

Y_predict1 = solve_svm_test(X_raw1, 
                           X_valid1, 
                           Y1, 
                           kernel='k_gram', 
                           k=6, lamb=0.01, gamma=3.)

train shape : (2000,)
train shape 2 : (2000,)
test shape : (1000,)
     pcost       dcost       gap    pres   dres
 0:  2.8983e-01  1.2948e+00  4e+03  1e+00  6e+04
 1:  1.2756e+00 -4.9209e+01  5e+01  2e-02  8e+02
 2:  1.1341e+00 -4.9220e+00  6e+00  2e-03  8e+01
 3:  9.7335e-01 -1.1897e+00  2e+00  5e-04  2e+01
 4:  6.1181e-01 -1.4378e-01  8e-01  1e-04  6e+00
 5:  2.0782e-01  1.1487e-01  9e-02  8e-06  3e-01
 6:  1.5178e-01  1.3215e-01  2e-02  1e-06  6e-02
 7:  1.4022e-01  1.3654e-01  4e-03  2e-07  8e-03
 8:  1.3784e-01  1.3747e-01  4e-04  1e-08  5e-04
 9:  1.3760e-01  1.3758e-01  2e-05  6e-10  2e-05
10:  1.3758e-01  1.3758e-01  7e-07  6e-10  4e-07
11:  1.3758e-01  1.3758e-01  2e-08  6e-10  6e-09
Optimal solution found.
accuracy on train : 0.997


In [38]:
print('train shape : {}'.format(X_raw2.shape))
print('train shape 2 : {}'.format(Y2.shape))
print('test shape : {}'.format(X_valid2.shape))

Y_predict2 = solve_svm_test(X_raw2, 
                           X_valid2, 
                           Y2, 
                           kernel='k_gram', 
                           k=6, lamb=1e-1, gamma=3.)

train shape : (2000,)
train shape 2 : (2000,)
test shape : (1000,)
     pcost       dcost       gap    pres   dres
 0:  6.1279e+00  7.7606e+00  4e+03  1e+00  1e+04
 1:  6.1775e+00 -1.6590e+02  2e+02  6e-02  5e+02
 2:  2.5770e+00 -3.1004e+01  3e+01  1e-02  9e+01
 3:  1.7643e+00 -2.5145e+00  4e+00  1e-03  9e+00
 4:  1.2983e+00  7.2710e-01  6e-01  1e-09  1e-08
 5:  8.6537e-01  7.8043e-01  8e-02  1e-09  2e-08
 6:  8.1592e-01  8.0095e-01  1e-02  8e-10  6e-09
 7:  8.0703e-01  8.0601e-01  1e-03  7e-10  1e-09
 8:  8.0645e-01  8.0642e-01  3e-05  7e-10  2e-10
 9:  8.0644e-01  8.0644e-01  7e-07  7e-10  3e-11
Optimal solution found.
accuracy on train : 0.8425


In [42]:
test0 = Y_predict0[:][0]
test1 = Y_predict1[:][0]
test2 = Y_predict2[:][0]

bound = np.concatenate((test0,test1,test2), axis=0).reshape((-1)).astype(int)
final = pd.DataFrame(np.arange(3000), columns=['Id'])
final['Bound'] = bound
final.to_csv('resultk_6.csv', index= None)

In [43]:
# Check before submit if the final has good shape
final

,Id,Bound
0,0,0
1,1,1
2,2,0
3,3,0
4,4,1
5,5,1
6,6,1
7,7,0
8,8,1
9,9,0
